In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys


import functions.parse_data as parse
import functions.handy_functions as hf

from py_torch_qrnn_adapt.neural_model import QuantileNetwork
from py_torch_qrnn_adapt.funcs import Sanity, Scenario1, Scenario2, Scenario3, Scenario4, Scenario5,\
                  MultivariateScenario1, MultivariateScenario2

from sklearn.metrics import mean_squared_error

In [5]:
channel_labels= ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06','Cloud_B07',
                 'Cloud_B08','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Cloud_B13']

data_water=parse.parse('cloudrm_water.dat')
data_clear=parse.parse('cloudrm_clear.dat')
data_ice=parse.parse('cloudrm_ice.dat')
data_mixed=parse.parse('cloudrm_mixed.dat')

#Concatinate all datasets
data_all=pd.concat([data_water, data_clear, data_ice, data_mixed])

In [6]:
data_all.index

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       49990, 49991, 49992, 49993, 49994, 49995, 49996, 49997, 49998, 49999],
      dtype='int64', length=200000)

In [7]:
data_all.Profile_ID.value_counts()

Profile_ID
6399    40
1489    38
9396    38
2979    38
428     37
        ..
4904     4
4834     4
4651     4
1808     4
4756     3
Name: count, Length: 10000, dtype: int64

In [8]:
channel_labels= ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06','Cloud_B07',
                 'Cloud_B08','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Cloud_B13']

data_water=parse.parse('cloudrm_water.dat')
data_clear=parse.parse('cloudrm_clear.dat')
data_ice=parse.parse('cloudrm_ice.dat')
data_mixed=parse.parse('cloudrm_mixed.dat')

#Concatinate all datasets
data_all=pd.concat([data_water, data_clear, data_ice, data_mixed])
data_all=data_all.drop(columns=['Surface_Desc','Cloud_B01','Clear_B01'])
data_all=hf.add_MSI_noise(data_all,channel_labels)

Noise standard deviation for Cloud_B02: 0.00335001428051948
Noise standard deviation for Cloud_B03: 0.002912530185416667
Noise standard deviation for Cloud_B04: 0.004058081082042254
Noise standard deviation for Cloud_B05: 0.0046524891611111115
Noise standard deviation for Cloud_B06: 0.007455351321348316
Noise standard deviation for Cloud_B07: 0.008871707484285717
Noise standard deviation for Cloud_B08: 0.04489677938000001
Noise standard deviation for Cloud_B09: 0.005688141120114942
Noise standard deviation for Cloud_B10: 0.003909328971491229
Noise standard deviation for Cloud_B11: 0.0014014724139999996
Noise standard deviation for Cloud_B12: 0.005030040539999999
Noise standard deviation for Cloud_B13: 0.004041267081999999


In [9]:
##Train test validation split##
X_labels= ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
           'Cloud_B07','Cloud_B08','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Cloud_B13',
           'Sat_Zenith_Angle','Sun_Zenith_Angle','Azimuth_Diff_Angle']

#Leave out 'GOT', 'Water_Vapor'
#Band 1 no go.
y_labels=['Clear_B02','Clear_B03','Clear_B04','Clear_B05','Clear_B06',
           'Clear_B07','Clear_B08','Clear_B09','Clear_B10','Clear_B11','Clear_B12','Clear_B13']

df=hf.normalise_input_df(data_all,X_labels)
df=hf.add_noise(df,X_labels,sigma=0.001)
num_epochs=2
batch_size=100

##Split data##
X=df[X_labels]
y=df[y_labels]

X_train, y_train, X_val, y_val, X_test, y_test=hf.split_data(X,y,split=[0.9,0.05,0.05])

In [11]:
quantiles=np.array([0.1,0.5,0.9])
batch_size=100
nepochs=50

model=QuantileNetwork(quantiles=quantiles)
model.fit(X_train.to_numpy(),y_train.to_numpy(),batch_size=batch_size,nepochs=nepochs)

ymax and min: tensor(22.4337) tensor(-3.1218)
Using marginal loss
Epoch 1


Batch number: : 1620it [00:55, 29.12it/s] 


			Validation Batch 0
			Validation Batch 100
			Saving test set results.      <----- New high water mark on epoch 1
Validation loss: 2.465773344039917 Best: 2.465773344039917
Epoch 2


Batch number: : 1620it [00:05, 323.74it/s]


			Validation Batch 0
			Validation Batch 100
			Saving test set results.      <----- New high water mark on epoch 2
Validation loss: 2.0341954231262207 Best: 2.0341954231262207
Epoch 3


...

			Validation Batch 0
			Validation Batch 100
Validation loss: 1.291732668876648 Best: 1.2815896272659302
Epoch 49


Batch number: : 1620it [00:04, 337.85it/s]


			Validation Batch 0
			Validation Batch 100
Validation loss: 1.3020122051239014 Best: 1.2815896272659302
Epoch 50


Batch number: : 1620it [00:04, 340.60it/s]


			Validation Batch 0
			Validation Batch 100
Decreasing learning rate to 0.00625
Validation loss: 1.2830452919006348 Best: 1.2815896272659302


In [35]:
preds = model.predict(X_test.to_numpy())

In [36]:
mean_squared_error(y_test.to_numpy(),preds[:,:,1])

0.004639387346144725

In [39]:
def PSNR(y_true,y_pred):
    mse = mean_squared_error(y_true,y_pred)
    maxval = np.amax(y_true)
    PSNR = 10*np.log10((maxval)**2/mse)
    
    return PSNR

In [40]:
PSNR(y_test,preds[:,:,1])

32.689449595993736

In [46]:
np.shape(y_test)[0]

19000

In [51]:
y_test_np=y_test.to_numpy()

In [54]:
def calc_outrate(y_test,preds):
    outcount = 0
    for i in range(np.shape(y_test_np)[0]):
        for j in range(np.shape(y_test_np)[1]):
            if y_test_np[i,j] < preds[i,j,0] or y_test_np[i,j] > preds[i,j,2]:
                outcount = outcount +1

    outrate = outcount/np.size(y_test)
    return outrate

In [52]:
outcount = 0
for i in range(np.shape(y_test_np)[0]):
    for j in range(np.shape(y_test_np)[1]):
        if y_test_np[i,j] < preds[i,j,0] or y_test_np[i,j] > preds[i,j,2]:
            outcount = outcount +1

outrate = outcount/np.size(y_test)

outrate

0.21011842105263157